<a href="https://colab.research.google.com/github/opencae/hpc-training-gpu-2023/blob/main/2_hpc_tc_cavity_benchmark_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「Google ColaboratoryのCPU・GPUを用いた3次元キャビティ・ベンチマークテスト実行」(演習) 「その2 GPU編」

オープンCAE学会 OpenFOAM GPUトレーニング2023 実行委員会 https://opencae.gitlab.io/web/page/training/opencae_hpc_training_gpu_2023/

## セルの実行と移動(再掲)
セルのコードを実行するには、セルをクリックして選択し、コードの左側にある実行ボタンをクリックします。

実行ボタンにマウスカーソルが乗ると表示されるように、「Commandキー+Enterキー」または「Ctrlキー+Enterキー」のキーボードショートカットによっても実行できます。

さらに、「Shiftキー+Enterキー」のキーボードショートカットによって、 セルのコードを実行(テキストの場合はフォーカス)した後に、次のセルに移動しますので、「Shiftキー+Enterキー」によって、効率的にノートブックの閲覧と実行を進めることができます。

## ハードウェア・アクセラレータとしてGPUを使用する設定
**再接続などで、既にGPU(T4)に接続されている場合には本設定は不要です。**

以下のようにして、ハードウェア・アクセラレータとしてGPUを使用する設定を行ないます。
 * メニュー→ランタイム→ランタイムのタイプを変更→ハードウェア アクセラレータ→T4 GPU→保存

万が一、Colabのリソース制限により「GPUバックエンドに接続できません」というメッセージが出た場合には、有料プランを契約するか、制限が解除された後で自習をお願いします。

* GPUバックエンドに接続できませんが出たのでColab Proに課金した話 https://misoji-engineer.com/archives/gpu-colab-pro.html

## リソースの確認

### GPUの確認
`nvidia-smi`(NVIDIA System Management Interface program)を用いて、
GPUの状況を確認することができます。

* watch nvidia-smiでGPU使用率などを確認・リアルタイムモニタリング https://note.nkmk.me/nvidia-smi-monitoring-gpu/
* nvidia-smi で GPU の使用状況を確認する方法 https://pystyle.info/nvidia-smi-usage/
* nvidia-smiの公式ドキュメント https://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf

#### GPUの状況確認

まずは`nvidia-smi`のヘルプを表示します。

In [ ]:
!nvidia-smi -h

引数無しで実行すると、システムに接続されているGPUの概要が表示されます。

In [ ]:
!nvidia-smi

上記は表形式なので、環境によっては表示が崩れて見にくいので、`-q`オプションを付けてGPUなどの情報を行形式で表示します。

In [ ]:
!nvidia-smi -q

#### システム内のGPU数

In [ ]:
!nvidia-smi --format=csv --query-gpu=count

#### GPUの公式な製品名

In [ ]:
!nvidia-smi --format=csv --query-gpu=name

#### 搭載されているGPUメモリの合計

In [ ]:
!nvidia-smi --format=csv --query-gpu=memory.total

#### プロパティ一覧の表示

In [ ]:
!nvidia-smi --help-query-gpu

### ソフトウェア

#### 標準でインストールされているCUDAコンパイラのバージョン確認

In [ ]:
!nvcc -V

#### 標準でインストールされているOpenMPIライブラリ

OpenMPIのインストール状況はompi_infoコマンドによって調べることができます。

* ompi_info(1) man page https://www.open-mpi.org/doc/current/man1/ompi_info.1.php

ompi_infoは多くの情報を出力しますが、-Vオプションでバージョンを表示します。

In [ ]:
!ompi_info -V

MPIライブラリが直接、GPUバッファへのsend/receiveを行うことができる機能であるCUDA-aware supportが有効化されているかは、以下のコマンドにより確認することができます。

* FAQ: Running CUDA-aware Open MPI https://www.open-mpi.org/faq/?category=runcuda#mpi-cuda-support

In [ ]:
!ompi_info --parsable --all | grep mpi_built_with_cuda_support:value

値が`false`となっているので、インストールされているOpenMPIは、CUDA-aware supportが有効化されていません。

## Google ドライブをマウントする

以下にGoogle ドライブをマウントする方法を示します。
Google のアカウントが必要になりますので、予め作成しておいてください。

Google ドライブをマウントには、下記のセルのコードを実行してください。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

既にマウント済みでない場合には、下記のようなメッセージが出ると思いますので、**Googleドライブに接続**を選択します。

---

**このノートブックに Google ドライブのファイルへのアクセスを許可しますか？**

このノートブックは Google ドライブ ファイルへのアクセスをリクエストしています。Google ドライブへのアクセスを許可すると、ノートブックで実行されたコードに対し、Google ドライブ内のファイルの変更を許可することになります。このアクセスを許可する前に、ノートブック コードをご確認ください。

スキップ  **Googleドライブに接続**

---


その後、アカウントの選択を求められた場合には、使用する**Googleアカウントを選択**します。

さらに、Googleアカウントへのアクセスをリクエストされた場合には、適宜**許可**します。

上記のようなgoogle の認証手順を踏んでGoogleドライブのマウントに成功すると、'Mounted at /content/drive'と出力され、Googleドライブが /content/drive/MyDrive にマウントされますので、以下を実行して、マウントされたディレクトリを表示してください。

In [ ]:
!ls /content/drive

'MyDrive/'と表示されれば問題ありません。
（'MyDriver/' だけではなく、 'Othercomputers/' などもあわせて表示される可能性があります。）

## OpenFOAM-v2306およびのインストールPETScライブラリ、external-solverモジュール
OpenFOAM-v2306のコンパイル済みのUbuntu向けパッケージと、前にビルドしたインストールPETScライブラリ、external-solverモジュールをインストールします。

なお、PETScのライブラリやexternal-solverモジュールのビルドは不要なので、ソースコードやビルドツールのパッケージのインストールも不要です。

### OepnFOAM-v2306の最小版パッケージのインストール

In [ ]:
!curl https://dl.openfoam.com/add-debian-repo.sh | bash;\
apt-get install openfoam2306

### Googleドライブへ保存したPETScライブラリおよびexternal-solverモジュールの解凍

OpenFOAMのトップディレクトリに移動します。

In [ ]:
%cd /usr/lib/openfoam/openfoam2306

ダミーファイルのThirdPartyを削除します。

In [ ]:
!rm ThirdParty

Googleドライブに保存したPETScライブラリとexternal-solverモジュールのライブラリのアーカイブファイルを展開します。

In [ ]:
!tar xf /content/drive//MyDrive/hpc-training-gpu-2023/petsc.tar.xz

##  RapidCFDのアーカイブ・ファイルのダウンロード
ColabのようにCPUの物理コア数が1、論理コア数が2つの環境でRapidCFDをビルドすると、数時間かかります。

そこで、ここではビルド済みのランタイムを含んだアーカイブファイルを、本トレーニング用の共有Googleドライブ( https://drive.google.com/drive/folders/16rZRqhRVnoEd91D-Onl_CjDtnT9dJtgk )に置きましたので、ここからダウンロードして、展開します。



### 共通ファイルと倍精度版バイナリのアーカイブ・ファイルのダウンロード
**各自のGoogleドライブにダウンロードするので、本ダウンロードは一度だけ行えば良いです。**

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1FIsejnVv7pLNwz42tDMGmR0cckN0r7Ee' -O /content/drive/MyDrive/hpc-training-gpu-2023/RapidCFD-dev.tar.xz

### 単精度版バイナリのアーカイブ・ファイルのダウンロード
**各自のGoogleドライブにダウンロードするので、本ダウンロードは一度だけ行えば良いです。**

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Vo3WXYkl2B3eR8TLhvlwzT9YNii0l8aO' -O /content/drive/MyDrive/hpc-training-gpu-2023/RapidCFD-dev_linux64NvccSPOpt.tar.xz

## RapidCFDのアーカイブ・ファイルの展開

### ホームディレクトリへの移動

In [ ]:
%cd

### RapidCFDディレクトリの作成

In [ ]:
!mkdir RapidCFD

### RapidCFDディレクトリへの移動

In [ ]:
%cd RapidCFD

### RapidCFDのアーカイブ・ファイルの展開

In [ ]:
!tar xf /content/drive/MyDrive/hpc-training-gpu-2023/RapidCFD-dev.tar.xz

### 単精度版バイナリのアーカイブ・ファイルの展開

In [ ]:
!tar xf /content/drive/MyDrive/hpc-training-gpu-2023/RapidCFD-dev_linux64NvccSPOpt.tar.xz

### 倍精度版icoFoamソルバの動作テスト


In [ ]:
!source /root/RapidCFD/RapidCFD-dev/etc/bashrc;\
icoFoam -help

icoFoamのヘルプ(Usage)が出力されることを確認してください。

標準のOpenFOAMの`icoFoam`と異なり、`-device`や`-devices` オプションがありますが、これは使用するGPUのIDを指定するオプションです。

今回ラインタイムのGPUは1つなので、これらのオプションは指定しません。

### 単精度版icoFoamソルバの動作テスト
単精度版のRapidCFDのソルバを動作させるには、環境設定時に`WM_PRECISION_OPTION=SP`を指定します。

もちろん、 /root/RapidCFD/RapidCFD-dev/etc/bashrc における `WM_PRECISION_OPTION`の設定行を直接変更しても良いのですが、普段使用しない設定については、環境設定時に直接変数を指定するほうが便利です。

In [ ]:
!source /root/RapidCFD/RapidCFD-dev/etc/bashrc WM_PRECISION_OPTION=SP;\
icoFoam -help

icoFoamのヘルプ(Usage)が出力されることを確認してください。

## キャビティベンチマークケースSの実行


### ケースディレクトリへの移動

In [ ]:
%cd /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/S

### GPUでのRapidCFD版icoFoamソルバの実行(ケースS-RapidCFD)


#### ケースファイル修正
 HPC TC委員会のベンチマークテストはOpenFOAM-v2006以降の新しいバージョン用であり、OpenFOAM-2.3.1をGPU向けに移植したRapicCFDでは実行できないので、ここではRapidCFDでも実行できるように設定を変更します。

また、短時間で実行できるように、2ステップのみ実行するように変更します。

なお、本ランタイムの再接続時など、既に設定が変更されている場合に備え、`git checkout`でオリジナルのファイルをチェックアウトしてから変更します。

In [ ]:
!case=S-RapidCFD;\
git checkout system/controlDict system/fvSchemes constant/transportProperties;\
sed -i -e 's|^\( *#includeFunc\)|//\1|' -e 's|\(endTime \).*|\1 0.002;|' system/controlDict;\
echo 'fluxRequired {default no;p;}' >> system/fvSchemes;\
sed -i s/'^\(nu\s*\[\)'/'nu \1'/ constant/transportProperties;\
sed s/PBiCGStab/PBiCG/ system/fvSolution.fixedNORM > system/fvSolution.$case;\
tail -n+1 constant/transportProperties system/{controlDict,fvSchemes,fvSolution.$case}

#### 倍精度版icoFoamソルバの実行

In [ ]:
!case=S-RapidCFD;\
cp system/fvSolution.$case system/fvSolution;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc;\
time icoFoam 2>&1 | tee log.icoFoam.$case

線形ソルバの前処理として、速度に対してはDILU(Diagonal-based Incomplete Lower-Upper)分解法が、圧力に対してはDIC(Diagonal-based Incomplete Cholesky)分解法が指定されているのにもかかわらず、 RapidCFDのソルバでのログでは、そのどちらもAINVになっていますが、これはRapidCFDがDILUやDICを実装しておらず、GPUでの実装が容易な前処理であるAINV(Approximate INVerse)法を強制的に用いるからです。
* DILU preconditioner https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-cg-preconditioner-dilu.html

* DIC preconditioner https://www.openfoam.com/documentation/guides/latest/doc/guide-solvers-cg-preconditioner-dic.html

* Labutin, Ilya B and Surodina, Irina V: Algorithm for Sparse Approximate Inverse Preconditioners in the Conjugate Gradient Method, Reliable Computing, vol.19, 2013 https://hgpu.org/?p=11066

* RapidCFD-dev/src/OpenFOAM/matrices/lduMatrix/preconditioners/DILUPreconditioner/DILUPreconditioner.C https://github.com/SimFlowCFD/RapidCFD-dev/blob/9fc614f4e816c51e5a718a1349cea9d72864042e/src/OpenFOAM/matrices/lduMatrix/preconditioners/DILUPreconditioner/DILUPreconditioner.C
  * *`Info<<"Using AINV preconditioner instead of DILU."<<endl;`*

* RapidCFD-dev/src/OpenFOAM/matrices/lduMatrix/preconditioners/DICPreconditioner/DICPreconditioner.C https://github.com/SimFlowCFD/RapidCFD-dev/blob/9fc614f4e816c51e5a718a1349cea9d72864042e/src/OpenFOAM/matrices/lduMatrix/preconditioners/DICPreconditioner/DICPreconditioner.C
  * *`Info<<"Using AINV preconditioner instead of DIC."<<endl;`*

#### 単精度版icoFoamソルバの実行


In [ ]:
!case=S-RapidCFD;\
cp system/fvSolution.$case system/fvSolution;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc WM_PRECISION_OPTION=SP;\
time icoFoam 2>&1 | tee log.icoFoam.$case.SP

#### 倍精度版ソルバに対する単精度版ソルバの加速率


In [ ]:
!case=S-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of single-precision solver compared to double-precision solver: ",(t[1]-t[0])/(t[3]-t[2])}' log.icoFoam.$case log.icoFoam.$case.SP

単精度は倍精度に比べて、線形ソルバの収束性が一般的に悪化するので、反復回数が大きく増えて実行時間が遅くなる場合もありますが、線形ソルバの反復回数が同程度の場合には、単精度のほうが通常実行が速くなります。

ただし、CPUでは倍精度版ソルバを用いているので、以降では倍精度版の実行時間を解析します。

#### 倍精度版での毎ステップの実行時間

In [ ]:
!case=S-RapidCFD;\
grep ExecutionTime log.icoFoam.$case

#### 倍精度版での1ステップの実行時間

In [ ]:
!case=S-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep: ",t[1]-t[0]}' log.icoFoam.$case

#### CPUのみを用いるOpenFOAM版icoFoamに対するGPUを併用するRapidCFD版icoFoamの加速率
なお、以下を実行するには、予めOpenFOAM版icoFoamを実行して、ログファイルlog.icoFoam.S-cpuが保存されている必要があります。

In [ ]:
!case=S-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of RapidCFD version icoFoam with GPU compared to OpenFOAM version icoFoam with CPU only: ",(t[1]-t[0])/(t[3]-t[2])}' log.icoFoam.S-cpu log.icoFoam.$case

加速率は約16となりました**(加速率は割当てられたCPUやGPUによって異なります)**。

#### GPUでのRapidCFD版icoFoamソルバのプロファイリング
時間がかかっているホットスポットを割り出すために、プロファイラ経由でソルバを実行します。
なお、新しいプロファイラであるnSightは標準ではインストールされていないので、
標準でインストールされているレガシーなプロファイラである`nvprof`を用います。

* Profiler User’s Guide https://docs.nvidia.com/cuda/profiler-users-guide/index.html
* NVIDIA Nsight Systems user guide. https://docs.nvidia.com/nsight-systems/UserGuide/index.html
* CUDAプログラミングをGoogle Colabで行う。 https://qiita.com/sakaia/items/3082752b5cfcc19537eb

In [ ]:
!case=S-RapidCFD;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc;\
time nvprof icoFoam 2>&1 | tee log.icoFoam.$case.nvprof

RapidCFDはThrustライブラリによるGPU実装なので、関数名が大変複雑となっていますが、`GPU activities:`から始まるホットスポットの行における`Name`から、`Foam::`で始まる本来のOpenFOAMの関数名を抜きだすと、主要なホットスポットは、`Foam::matrixMultiplyFunctor`と`Foam::AINVPreconditionerFunctor`であり、計算時間の割合は共に約30%となっています。

* Foam::matrixMultiplyFunctor
  * RapidCFD-dev/src/OpenFOAM/matrices/lduMatrix/lduMatrix/lduMatrixATmul.C https://github.com/SimFlowCFD/RapidCFD-dev/blob/master/src/OpenFOAM/matrices/lduMatrix/lduMatrix/lduMatrixATmul.C
* Foam::AINVPreconditionerFunctor
  * RapidCFD-dev/src/OpenFOAM/matrices/lduMatrix/preconditioners/AINVPreconditioner/AINVPreconditionerF.H
 https://github.com/SimFlowCFD/RapidCFD-dev/blob/master/src/OpenFOAM/matrices/lduMatrix/preconditioners/AINVPreconditioner/AINVPreconditionerF.H
* 出川智啓 : GPU最適化ライブラリの利用(その3)、長岡技術科学大学2015年度GPGPU講習会 https://www.slideshare.net/ssuserf87701/gpgpu-seminar-gpu-accelerated-libraries-3-of-3-thrust

### PETScライブラリを用いたGPU併用でのicoFoamソルバの実行(ケースS-petsc-1)
PETScライブラリを用いて、圧力方程式用のCG法線形ソルバをGPUで動かします。

#### ケースファイル修正




3次元キャビティベンチマークテストにおける格子数が64MのXLケースでは、PETScのCG法線形ソルバを使う設定ファイルがあるので、この設定を流用します。
ただし、GPUのCuSPARSEライブラリを用いて動作させるよう、`mat_types`に`aijcusparse`を指定します。

* MATAIJCUSPARSE https://petsc.org/release/manualpages/Mat/MATAIJCUSPARSE/
* 出川智啓 : GPU最適化ライブラリの利用(その2)、長岡技術科学大学2015年度GPGPU講習会 https://www.slideshare.net/ssuserf87701/gpgpu-seminar-gpu-accelerated-libraries-2-of-3-cusparse

また、速度の線形ソルバを、RapidCFD等の他の解析での設定と合わせて`PBiCG`に変更します。

このケースを`S-petsc-1`として、これらの修正を行なった線形ソルバの設定を`system/fvSolution.S-petsc-1`に保存しておきます。

In [ ]:
!case=S-petsc-1;\
sed -e s/mpiaijmkl/aijcusparse/ -e s/PBiCGStab/PBiCG/ ../XL/system/fvSolution.PETSc-ICC-CG.fixedNORM | tee system/fvSolution.$case

#### PETScライブラリを用いたicoFoamソルバの実行
まず、線形ソルバの設定`system/fvSolution.ケース名`を`system/fvSolution`にコピーします。

external-solverモジュールのビルド時にも出力されていましたが、 PETScライブラリを有効にするには、`eval $(foamEtcFile -sh -config petsc -- -force)`を実行します。

また、以前確認したように、インストールされているOpenMPIはGPU-aware supportが有効になっていないので、`PETSC_OPTIONS`の環境変数に`-use_gpu_aware_mpi 0`を指定して、`icoFoam`ソルバを実行します。

さらに、ソルバ実行時にexternal-moduleモジュールのpetscFoamライブラリを読むように、`icoFoam`コマンドに`-lib petscFoam`オプションを指定します。

なお、`-lib petscFoam`オプションを指定せずに、`system/controlDict`に`libs (petscFoam);`の設定をしてもpetscFoamライブラリを読みますが、この設定ではRapidCFDでの実行ができなくなるので、ここでは`icoFoam`コマンドにオプションを指定する方法を用います。

In [ ]:
!case=S-petsc-1;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0' icoFoam -lib petscFoam 2>&1 | tee log.icoFoam.$case

`Initializing PETSc... success`のログにより、PETScライブラリを用いた初期化ができていることがわかります。

また、圧力の線形ソルバ名が`PETSc-cg`となっており、PETScのcgソルバが用いられていることが確認できます。

#### 毎ステップの実行時間

In [ ]:
!case=S-petsc-1;\
grep ExecutionTime log.icoFoam.$case

2ステップ分の2回ではなく、PETScの初期化の前後でも出力され、4回分出力されています。

#### 1ステップの実行時間
4番目と3番目の実行時間の差を計算します。

In [ ]:
!case=S-petsc-1;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep: ",t[3]-t[2]}' log.icoFoam.$case

#### 圧力方程式に対するCG法線形ソルバをPETScライブラリを用いてGPUで実行した場合の加速率


In [ ]:
!case=S-petsc-1;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of PETSc CG solver with GPU compared to OpenFOAM CG solver with CPU only: ",(t[1]-t[0])/(t[5]-t[4])}' log.icoFoam.S-cpu log.icoFoam.$case

加速率はRapidCFDが約16であるのに対し、今回は約1.2とだいぶ低くなりましたが、 圧力線形ソルバの反復回数が増大していることも、一つの要因として考えられます**(加速率は割りあてられたCPUとGPUによって異なります)**。

In [ ]:
!cat log.icoFoam.S-cpu

実際、上記の全CPU実行での`icoFoam`ソルバのログからわかるとおり、CPU実行では圧力線形ソルバの反復回数は約250回でしたが、GPU実行では2100回と約8倍以上増加しています。

RapidCFDでも約380回とCPU実行に比べて増えていますが、それでも1.5倍程度です。

どのケースでも線形ソルバはCG法ですので、その他の大きな違いとしては前処理があげられます。

そこで、最後のカスタマイズ演習では、3次元キャビティベンチマーク標準のPETScの線形ソルバの設定をカスタマイズすることで、さらに高速化できるか試してみます。

#### PETScのログオプション付き実行
`PETSC_OPTIONS`に`-log_view`を付けると、PETScに関するプロファイリング情報等が出力されます。

* Understanding and Tuning Performance in PETSc (on emerging many core,GPGPU, and traditional architectures) https://www.mcs.anl.gov/petsc/meetings/2019/slides/mills-performance-gpus-tutorial-petsc2019.pdf

* PETSc/TAO Users Manual, 4.3.1 Basic Profiling Information, https://petsc.org/release/docs/manual/manual.pdf

In [ ]:
!case=S-petsc-1;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0 -log_view' icoFoam -lib petscFoam 2>&1 | tee log.icoFoam.$case.log_view

#### プロファイラ系由での実行
RapidCFDと同様に`nvprof`経由で`icoFoam`を実行することにより、ホットスポットのようなプロファイル情報が出力されますが、`icoFoam`全体ではなく、あくまでGPUに関するものが出力されます。

In [ ]:
!case=S-petsc-1;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0' nvprof icoFoam -lib petscFoam 2>&1 | tee log.icoFoam.petsc-nvprof

`GPU activities`のホットスポットは 約62%を占める`cusparse::spsm_v1_ker` です。

* cuSPARSE 6.6.8. cusparseSpSM() https://docs.nvidia.com/cuda/cusparse/index.html?highlight=spsm#cusparsespsm
  * *The function solves a system of linear equations whose coefficients are represented in a sparse triangular matrix:*

## キャビティベンチマークケースMの実行


### ケースディレクトリへの移動

In [ ]:
%cd /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/M

### GPUでのRapidCFD版icoFoamソルバ実行(ケースM-RapidCFD)

#### ケースファイル修正
ケースSと同様に修正します。

In [ ]:
!case=M-RapidCFD;\
git checkout system/controlDict system/fvSchemes constant/transportProperties;\
sed -i -e 's|^\( *#includeFunc\)|//\1|' -e 's|\(endTime \).*|\1 0.0005;|' system/controlDict;\
echo 'fluxRequired {default no;p;}' >> system/fvSchemes;\
sed -i s/'^\(nu\s*\[\)'/'nu \1'/ constant/transportProperties;\
sed s/PBiCGStab/PBiCG/ system/fvSolution.fixedNORM > system/fvSolution.$case;\
tail -n+1 constant/transportProperties system/{controlDict,fvSchemes,fvSolution.$case}

#### 倍精度版icoFoamソルバの実行

In [ ]:
!case=M-RapidCFD;\
cp system/fvSolution.$case system/fvSolution;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc;\
time icoFoam 2>&1 | tee log.icoFoam.$case

#### 単精度版icoFoamソルバの実行


In [ ]:
!case=M-RapidCFD;\
cp system/fvSolution.$case system/fvSolution;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc WM_PRECISION_OPTION=SP;\
time icoFoam 2>&1 | tee log.icoFoam.$case.SP

#### 倍精度版ソルバに対する単精度版ソルバの加速率


In [ ]:
!case=M-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of single-precision solver compared to double-precision solver: ",(t[1]-t[0])/(t[3]-t[2])}' log.icoFoam.$case log.icoFoam.$case.SP

#### 倍精度版での毎ステップの実行時間

In [ ]:
!case=M-RapidCFD;\
grep ExecutionTime log.icoFoam.$case

#### 倍精度版での1ステップの実行時間

In [ ]:
!case=M-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep: ",t[1]-t[0]}' log.icoFoam.$case

#### CPUのみを使用するOpenFOAM版icoFoamに対するGPUを併用するRapidCFD版icoFoamの加速率


In [ ]:
!case=M-RapidCFD;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of RapidCFD version icoFoam with GPU compared to OpenFOAM version icoFoam with CPU only: ",(t[1]-t[0])/(t[3]-t[2])}' log.icoFoam.S-cpu log.icoFoam.$case

加速率は約18となりました**(加速率は割当てられたGPUとGPUによって異なります)**。

#### GPUでのRapidCFD版icoFoamソルバのプロファイリング

In [ ]:
!case=M-RapidCFD;\
source /root/RapidCFD/RapidCFD-dev/etc/bashrc;time nvprof icoFoam 2>&1 | tee log.icoFoam.$case.nvprof

このケースでも、主要なホットスポットは、`Foam::AINVPreconditionerFunctor`と、`Foam::matrixMultiplyFunctor`であり、計算時間は共に30%強になっています**(割当てられたGPUによって異なります)**。

### PETScライブラリを用いたGPU併用でのicoFoamソルバの実行(ケースM-petsc-1)



#### ケースファイル修正
ケースS同様に修正しますが、ケースMでは圧力の線形ソルバが3000回の反復でも収束しないので、最大反復数を10000に変更します。

In [ ]:
!case=M-petsc-1;\
sed -e s/'\(mat_type \)mpiaijmkl'/'\1aijcusparse'/ -e s/PBiCGStab/PBiCG/ -e s/'\(maxIter *\)3000;'/'\110000;'/ ../XL/system/fvSolution.PETSc-ICC-CG.fixedNORM | tee system/fvSolution.$case

#### PETScライブラリを用いたicoFoamソルバの実行


In [ ]:
!case=M-petsc-1;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0' icoFoam -lib petscFoam 2>&1 | tee log.icoFoam.$case

#### 毎ステップの実行時間

In [ ]:
!case=M-petsc-1;\
grep ExecutionTime log.icoFoam.$case

#### 1ステップの実行時間


In [ ]:
!case=M-petsc-1;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep: ",t[3]-t[2]}' log.icoFoam.$case

#### 圧力方程式に対するCG法線形ソルバをPETScライブラリを用いてGPUで実行した場合の加速率


In [ ]:
!case=M-petsc-1;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of PETSc CG solver with GPU compared to OpenFOAM CG solver with CPU only: ",(t[1]-t[0])/(t[5]-t[4])}' log.icoFoam.M-cpu log.icoFoam.$case

圧力方程式のCG法線形ソルバをCPUで実行する場合と変わらない結果となっていますが、ケースSと同様に圧力線形ソルバの反復回数が増大していることも、原因と一つとして考えられます。

In [ ]:
!cat log.icoFoam.M-cpu

実際、上記の全CPU実行でのicoFoamソルバのログからわかるとおり、CPU実行では圧力線形ソルバの反復回数は約510回でしたが、GPU実行では4300回と約8倍以上増加しています。

## カスタマイズ演習
本演習のカスタマイズ演習では計算時間の関係でケースSのみ扱います。

### PETScライブラリを用いたGPU併用計算の加速率
**(加速率は割当てられたGPUとGPUによって異なります)**

PETScライブラリを用いてGPU併用のicoFoamソルバ実行を行なったケースS-petsc-1の加速率は、CPUのみを使用したケースS-cpuに比べて約1.2と、RapidCFDの約16に比べ低くなっています。

さらに、ケースMでは、PETScライブラリのGPU併用とCPU実行とで実行時間がほぼ変わらず、RapidCFDの加速率約18と対照的です。

### 線形ソルバの前処理
ただし、CPUの解析では、圧力や速度に関する線形ソルバの前処理に、それぞれDIC(Diagonal-based Incomplete Cholesky)法やDILU(Diagonal-based Incomplete Lower-Upper)法を使っているのに対し、 RapidCFDではどちらもAINV(Approximate INVerse)法を使用するなど、GPUに合わせて計算条件の変更を行なっています。

一方、PETScを用いたケースS-petsc-1では、線形ソルバの設定は以下となっています。
なお、M-petsc-1も圧力に関する最大反復数が10000になっている以外は同じです。

In [ ]:
!cat /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/S/system/fvSolution.S-petsc-1

ここで、`petsc/options`辞書がPETScの線形ソルバの設定ですが、圧力に関する線形ソルバ`ksp_type`(Krylov Subspace method and a Preconditioner TYPE)は、元々の設定同様にcg(CG)です。

* KSP: Linear System Solvers https://petsc.org/release/manual/ksp/

しかし、前処理(`pc_type`)(PreConditioner TYPE)はbjacobi(Block-Jacobi)であり、
副前処理(`sub_pc_type`)が`icc`(InComplete Cholesky)になっています。

* Preconditioners https://petsc.org/release/manual/ksp/#sec-pc
* PCBJACOBI https://petsc.org/release/manualpages/PC/PCBJACOBI/
* PCICC Incomplete Cholesky factorization preconditioners. https://petsc.org/release/manualpages/PC/PCICC/
* 前処理手法について http://nkl.cc.u-tokyo.ac.jp/15n/SolverPrecond.pdf

RapidCFDでは、前処理はIncomplete Cholesky系のDICではないのですが、GPU向けに実装が容易(並列実行が容易)、かつ、中強度の前処理(反復回数が中程度に減少する)であるAINVを用いており、実際にケースSでは、圧力線形ソルバの反復回数の増加は、強い前処理のDICに比べても1.5倍に抑えられていました。

従って、PETScでも、前処理を本問題により適するものに変更することで、より高速化される可能性があります。

さらに、PETScでは、`petsc/caching`において、係数行列や前処理行列の更新に関する設定が可能ですので、これらの設定を変更することで、より高速化する可能性があります。

また、速度の線形ソルバは`PBiCG`なので、これは従来通りCPUで動作していますが、これもPETScを用いてGPUで動作されることが可能です。




### ケースディレクトリへの移動

In [ ]:
%cd /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/S

### PETScのヘルプ出力
`PETSC_OPTIONS`に`-help`を付けてソルバを実行すると、PETScのヘルプが出力されますが、ここにはPETScの様々な設定と設定候補の一覧も出力されますので、参考になります。

なお、PETScのヘルプを出力するのが目的であり、`icoFoam`を最後まで実行する必要がないので、`icoFoam`のオプションに`-dry-run`(Check case set-up only using a single time step)を付けています。

In [ ]:
!case=S-petsc-1;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0 -help' icoFoam -lib petscFoam -dry-run 2>&1 | tee log.icoFoam.$case.help

上記のログから変更したいオプションを検索したり、以下のように`grep`コマンドを用いてログから抜き出します。

In [ ]:
!case=S-petsc-1;\
grep -e pc_type -e ksp_cg_single_reduction -e sub_pc_factor_shift_type log.icoFoam.$case.help

上記のペルプメッセージや以下の資料等を参考に、`system/fvSolution`の設定を変更しますが、特にZampiniらによる発表スライドでの設定例や最後のページが参考になります。

* S. Zampini, S. Bnà, M.Valentini, I. Spisso : GPU-accelerated OpenFOAM simulations using PETSc4FOAM
  * 発表スライド https://wiki.openfoam.com/images/c/cd/OpenFOAM_2020_KAUST_Zampini.pdf
  * 論文 https://www.esi-group.com/sites/default/files/resource/other/1806/8th_OpenFOAM_Conference_Cineca_Spisso_2_0.pdf
* Summary of Sparse Linear Solvers Available In PETSc https://petsc.org/release/overview/linear_solve_table/
* KSP: Linear System Solvers https://petsc.org/release/manual/ksp/
* Preconditioners https://petsc.org/release/manual/ksp/#sec-pc
* PCBJACOBI https://petsc.org/release/manualpages/PC/PCBJACOBI/
* PCICC Incomplete Cholesky factorization preconditioners. https://petsc.org/release/manualpages/PC/PCICC/
* KSPBICG https://petsc.org/release/manualpages/KSP/KSPBICG/

また、AMDがLid_driven_cavity-3Dベンチマークテスト用のDockerやSingularityのイメージを以下で公開していますが、その中での`system/fvSolution`の内容の一部も転載しておきますので、参考にしてください。

 * AMD OpenFOAM https://www.amd.com/en/technologies/infinity-hub/openfoam

```
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2006                                 |
|   \\  /    A nd           | Website   www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      fvSolution;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

solvers
{
    p
    {
        solver petsc;

        petsc
        {
            options
            {
                ksp_type  cg;
                ksp_cg_single_reduction  true;
                ksp_norm_type none;
                //mat_type aijcusparse; // CUSPARSE
                //mat_type aijkokkos; // KOKKOS
                        mat_type mpiaijhipsparse; // HIPSPARSE

                //preconditioner
                pc_type gamg;
                pc_gamg_type "agg"; // smoothed aggregation
                pc_gamg_agg_nsmooths "1"; // number of smooths for smoothed aggregation (not smoother iterations)
                pc_gamg_coarse_eq_limit "100";
                pc_gamg_reuse_interpolation true;
                pc_gamg_aggressive_coarsening "2"; //square the graph on the finest N levels
                pc_gamg_threshold "-1"; // increase to 0.05 if coarse grids get larger
                pc_gamg_threshold_scale "0.5"; // thresholding on coarse grids
                pc_gamg_use_sa_esteig true;

                // mg_level config
                mg_levels_ksp_max_it "1"; // use 2 or 4 if problem is hard (i.e stretched grids)
                        mg_levels_esteig_ksp_type cg; //max_it "1"; // use 2 or 4 if problem is hard (i.e stretched grids)

                // coarse solve (indefinite PC in parallel with 2 cores)
                mg_coarse_ksp_type "gmres";
                mg_coarse_ksp_max_it "2";

                // smoother (cheby)
                mg_levels_ksp_type chebyshev;
                mg_levels_ksp_chebyshev_esteig "0,0.05,0,1.1";
                mg_levels_pc_type "jacobi";
            }
            caching
            {
                // PISO with 2 correctors
                matrix
                {
                    update periodic;

                    periodicCoeffs
                    {
                        frequency  2;
                    }
                }
                // basically setup the preconditioner once
                // and reuse it for 40 consecutive solves
                preconditioner
                {
                    //update always;
                    update periodic;

                    periodicCoeffs
                    {
                        frequency  40;
                    }
                }
            }
        }
        tolerance       1.e-04;
        relTol          0;
        maxIter         3000;
    }

    pFinal
    {
        $p;
        relTol          0;
    }

    U
    {
        solver petsc;
        petsc
        {
            options
            {
                ksp_type bcgs;
                ksp_norm_type none;
                        mat_type mpiaijhipsparse; //other options -aijkokkos, aijcusparse, etc.
                pc_type bjacobi;
                sub_pc_type ilu;
            }
            use_petsc_residual_norm true; // we only run 5 iterations, do not use L1 residual test
        }
        tolerance       0;
        relTol          0;
        maxIter         5;
    }
}

PISO
{
    nCorrectors     2;
    nNonOrthogonalCorrectors 0;
    pRefCell        0;
    pRefValue       0;
}
```

さらに、external-solverモジュールのチュートリアルケースの設定ファイルも参考になります。

* tutorials https://develop.openfoam.com/modules/external-solver/-/tree/develop/tutorials

pipeOneD1ケースの `fvSolution-petsc-gamg-device`に記載されているように、`ksp_view`に"`::ascii_info"`を指定すると、線形ソルバの詳細が実行時に表示されるので、設定確認に有用なので、試してみてください。

https://develop.openfoam.com/modules/external-solver/-/blob/develop/tutorials/basic/laplacianFoam/pipeOneD/system/fvSolution-petsc-gamg-device
```
        petsc
        {
            options
            {
               //ksp_view "::ascii_info"; // uncomment to list details of the solve
```

### PETScライブラリを用いたGPU併用でのicoFoamソルバのカスタマイズ実行(ケースS-petsc-2)
ケースS-petsc-1の線形ソルバの設定をカスタマイズしたケースS-petsc-2を作成し実行します。

#### ケースS-petsc-1の線形ソルバの設定コピー



In [ ]:
!case=S-petsc-2;\
cp system/fvSolution.S-petsc-1 system/fvSolution.$case

#### ケースS-petsc-2の線形ソルバの設定変更
左のツールバーにおけるフォルダーのマークを押すと、ファイル・ブラウザになりますので、
フォルダを辿って、 /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/S/system/fvSolution.S-petsc-2 まで行ってダブルクリックするか、本ファイルへのリンクをクリックします。

すると、右にファイルの編集ウィンドウが表示されるので、適宜編集します。

なお、編集後にマウスのフォーカスが編集ウィンドウから外れると、ファイル名が編集されている事を示すアスタリスクが消え、ファイルが保存されます。

#### PETScライブラリを用いたicoFoamソルバの実行
変更した線形ソルバの設定を用いて、`icoFoam`ソルバの実行します。

In [ ]:
!case=S-petsc-2;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
eval $(foamEtcFile -sh -config petsc -- -force);\
PETSC_OPTIONS='-use_gpu_aware_mpi 0' icoFoam -lib petscFoam 2>&1 | tee log.icoFoam.$case

#### 圧力方程式に対するCG法線形ソルバをPETScライブラリを用いてGPUで実行した場合の加速率


In [ ]:
!case=S-petsc-2;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Speedup of PETSc CG solver with GPU compared to OpenFOAM CG solver with CPU only: ",(t[1]-t[0])/(t[5]-t[4])}' log.icoFoam.S-cpu log.icoFoam.$case

ケースS-petsc-1に比べて高速化されましたでしょうか?

S-petsc-2と同様に他の設定も検討してみてください。

なお、セルは複数コピーできますので、「PETScライブラリを用いたGPU併用でのicoFoamソルバのカスタマイズ実行(ケースS-petsc-2)」のセクションを折りたたんだ上でセルをコピー&ペーストすると、他のケースを検討しやすいと思います。

## その他の参考文献

* Simone Bnà, Ivan Spissoa, Mark Olesenb, Giacomo Rossic : A Library to plug-in PETSc into the OpenFOAM Framework https://prace-ri.eu/wp-content/uploads/WP294-PETSc4FOAM-A-Library-to-plug-in-PETSc-into-the-OpenFOAM-Framework.pdf
* OpenFOAM Conference - High Performance Computing https://www.esi-group.com/openfoam-conference/openfoam-tracks/openfoam-conference-high-performance-computing
* How to use Valgrind to profile OpenFOAM solvers https://solution.esi.co.jp/openfoam/blog/how-to-use-valgrind-to-profile-openfoam-solvers
* GPU Support in PETSc 3.10
 https://www.mcs.anl.gov/~oanam/NASA2019/GPUslides.pdf
* cuSPARSE Library https://docs.nvidia.com/cuda/archive/10.1/pdf/CUSPARSE_Library.pdf
* Question about petsc4Foam performance https://develop.openfoam.com/modules/external-solver/-/issues/18
* Stefano Zampini : The Portable Extensible Toolkit for Scientific Computing Overview and ECRC success stories https://www.hpc.kaust.edu.sa/sites/default/files/files/public/Shaheen_training/NLHPC_2020/PETSc_Zampini2020.pdf
* PETSC gives a slowdown instead of speedup (GAMG) https://www.cfd-online.com/Forums/openfoam-programming-development/234081-petsc-gives-slowdown-instead-speedup-gamg.html
* AMGX GPU SOLVER DEVELOPMENTS FOR OPENFOAM https://wiki.openfoam.com/images/a/a4/OpenFOAM_2020_NVIDIA_Martineau.pdf
* Missing header file for amgxwrapper https://develop.openfoam.com/modules/external-solver/-/issues/25
* OpenCL porting? #46 https://github.com/SimFlowCFD/RapidCFD-dev/issues/46
* 今野 雅 : Xeon Phi KNLプロセッサを搭載したスーパーコンピュータOakforest-PACSにおけるOpenFOAMの高速化, オープンCAEシンポジウム2018 https://www.slideshare.net/MasashiImano/xeon-phi-knloakforestpacsopenfoam
* Wisteria/BDEC-01および「不老」でのOpenFOAM/RapidCFDベンチマークテスト, 第85回オープンCAE勉強会@関西 https://gitlab.com/masaz/OpenCAEWorkshop/-/raw/master/OpenCAEWorkshopAtKansai20210627/doc/OpenCAEWorkshopAtKansai20210627.pdf?inline=false


**以上で、「その2 GPU編」のノートブックは終了です。**